<a href="https://colab.research.google.com/github/Mreeb/llama2-Fine-tuning-On-Custom-Medical_data/blob/master/Fine_Tuning_Llama2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing the Required Packages


In [2]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

#Step 2: Import All the Required Libraries

In [3]:
import os
import random
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from transformers import pipeline

In [4]:
system_message  = "Hello! I'm here to provide concise information about various skin diseases, including their causes, symptoms, treatments, and recommended medications. How can I assist you today?"
print(system_message)

Hello! I'm here to provide concise information about various skin diseases, including their causes, symptoms, treatments, and recommended medications. How can I assist you today?


In [5]:
import pandas as pd
csv_file_path = 'combined_data.csv'
df = pd.read_csv(csv_file_path)
print('Here are the first few rows of the DataFrame:')
df.head(10)

Here are the first few rows of the DataFrame:


,prompt,response,prompt_word_count,response_word_count
0,What is psoriasis and what are its common symp...,Psoriasis is a chronic autoimmune condition th...,9,67
1,What is the etiology of acne?,Acne is primarily caused by the overproduction...,6,64
2,What are the recommended medications for atopi...,There are several medications available for th...,8,83
3,Can you tell me about the treatment modalities...,Melanoma treatment depends on the stage and lo...,10,101
4,What is rosacea and what are its common symptoms?,Rosacea is a common skin condition that causes...,9,86
5,What causes seborrheic dermatitis?,The exact cause of seborrheic dermatitis is un...,4,68
6,What are the recommended medications for treat...,Treatment for vitiligo aims to stop or slow th...,8,67
7,What are the treatment options for basal cell ...,Treatment options for basal cell carcinoma dep...,9,88
8,What is eczema and what are its common symptoms?,"Eczema, also known as atopic dermatitis, is a ...",9,96
9,What causes contact dermatitis?,Contact dermatitis is caused by a direct react...,4,67


In [6]:
df = df.drop("prompt_word_count", axis = 1)
df = df.drop("response_word_count", axis = 1)

In [7]:
df.head()

,prompt,response
0,What is psoriasis and what are its common symp...,Psoriasis is a chronic autoimmune condition th...
1,What is the etiology of acne?,Acne is primarily caused by the overproduction...
2,What are the recommended medications for atopi...,There are several medications available for th...
3,Can you tell me about the treatment modalities...,Melanoma treatment depends on the stage and lo...
4,What is rosacea and what are its common symptoms?,Rosacea is a common skin condition that causes...


In [10]:
train_df = df.sample(frac=0.9, random_state=42)
test_df = df.drop(train_df.index)

In [11]:
train_df.to_json('train.jsonl', orient='records', lines=True)
test_df.to_json('test.jsonl', orient='records', lines=True)

In [12]:
model_name = "NousResearch/Llama-2-7b-chat-hf"
dataset_name = "/content/train.jsonl"

new_model = "Medi-llama-2-7b-custom1000"
lora_r = 64
lora_alpha = 16
lora_dropout = 0.1
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False

output_dir = "./results"

num_train_epochs = 5

fp16 = False
bf16 = False

per_device_train_batch_size = 4

per_device_eval_batch_size = 4
gradient_accumulation_steps = 1

gradient_checkpointing = True

max_grad_norm = 0.3


learning_rate = 2e-4
weight_decay = 0.001

optim = "paged_adamw_32bit"
lr_scheduler_type = "cosine"

max_steps = -1

warmup_ratio = 0.03

group_by_length = True

save_steps = 0

logging_steps = 25

max_seq_length = None
packing = False

device_map = {"": 0} ## LOAD THE ENTIRE MODLE ON THE GPU

In [13]:
# Load datasets
train_dataset = load_dataset('json', data_files='/content/train.jsonl', split="train")
valid_dataset = load_dataset('json', data_files='/content/test.jsonl', split="train")

# Preprocess datasets
train_dataset_mapped = train_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\n' + prompt + ' [/INST] ' + response for prompt, response in zip(examples['prompt'], examples['response'])]}, batched=True)
valid_dataset_mapped = valid_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\n' + prompt + ' [/INST] ' + response for prompt, response in zip(examples['prompt'], examples['response'])]}, batched=True)

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_mapped,
    eval_dataset=valid_dataset_mapped,  # Pass validation dataset here
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)



Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1314 [00:00<?, ? examples/s]

...

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/1314 [00:00<?, ? examples/s]

Map:   0%|          | 0/146 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
25,1.758300
50,1.120000
75,0.808300
100,0.697800
125,0.749600
150,0.653100
175,0.728500
200,0.617700
225,0.697700
250,0.610500


## Testing the Fine Tuned Lamma2 Medical LLM


In [ ]:
logging.set_verbosity(logging.CRITICAL)
prompt = f"[INST] <<SYS>>\n{system_message}\n<</SYS>>\n\nGive Me Medication for diaper Rash? [/INST]" # replace the command here with something relevant to your task
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=500)
result = pipe(prompt)
print(result[0]['generated_text'])

In [ ]:
model

#Step 10: Run Inference on Original LLama 2 Model

In [ ]:
prompt = f"[INST] <<SYS>>\n{system_message}\n<</SYS>>\n\nWhat is siroline syrup used for? [/INST]"
num_new_tokens = 200

num_prompt_tokens = len(tokenizer(prompt)['input_ids'])
max_length = num_prompt_tokens + num_new_tokens

gen = pipeline('text-generation', model=model, tokenizer=tokenizer, max_length=max_length)
result = gen(prompt)
print(result[0]['generated_text'].replace(prompt, ''))

In [14]:
del model

In [13]:
del pipe

NameError: ignored

In [15]:
del trainer

In [16]:
import gc
gc.collect()
gc.collect()

0

In [18]:
import torch

# Release GPU memory
torch.cuda.empty_cache()

In [ ]:
from google.colab import drive

In [19]:
# Merge and save the fine-tuned model

#drive.mount('/content/drive')

#model_path = "/content/drive/MyDrive/llama-2-7b-custom"  # change to your preferred path

# Reload model in FP16 and merge it with LoRA weights

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Save the merged model
#model.save_pretrained(model_path)
#tokenizer.save_pretrained(model_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Run This cell If You want to Save the Fine Tuned LLM To Drive


In [ ]:
drive.mount('/content/drive')

model_path = "/content/drive/MyDrive/llama-2-7b-custom100-FineTuned"  # change to your preferred path

# Save the merged model
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

#Step 12: Load the Fine Tuned Model and Run Inference

In [ ]:
from google.colab import drive
from transformers import AutoModelForCausalLM, AutoTokenizer

drive.mount('/content/drive')

model_path = "/content/drive/MyDrive/Medi-llama-2-7b-custom100"  # change to the path where your model is saved

model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [ ]:
from transformers import pipeline

prompt = "Give me Medication For Acne cystic it is Not Severe?"  # change to your desired prompt
gen = pipeline('text-generation', model=model, tokenizer=tokenizer)
result = gen(prompt)
print(result[0]['generated_text'])

# Pushing the model TO Hub

In [20]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [22]:
!huggingface-cli login

model.push_to_hub("Mreeb/Medi-llama-2-7b-custom1000", check_pr=True)

tokenizer.push_to_hub("Mreeb/Medi-llama-2-7b-custom1000",check_pr=True)


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-crede

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Mreeb/Medi-llama-2-7b-custom1000/commit/4f60491e977fcf6a3e78c1c2dee49cc3e288ca83', commit_message='Upload tokenizer', commit_description='', oid='4f60491e977fcf6a3e78c1c2dee49cc3e288ca83', pr_url=None, pr_revision=None, pr_num=None)